# **Evolution of Cellular Automata Rules with GE**

*Student*: Federico Pellizzaro

*Matricola*: IN2000196

*Project Target*: Evolve the local rule of a CA to conform to a given behavior. In other words, given a target matrix which could have a certain shape/pattern/configuration, evolve the CA rules in order to get the output matrix as similar to the target matrix.

The inspiration for this project came from the very well known game [*Conway's Game of Life*](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life).

All classes that are shown below could have been moved to a separate python file (for example `utils.py`) but for semplicity reasons I've decided to include them here.

Useful libraries that will be used in the project

* `re` it's a library used to perform regular expression searches
* `scipy.spatial.distance` has been used to compute the jaccard distance
* `random`, `numpy`, `copy` used for regular usage

In [ ]:
import re
import random
import numpy as np
import copy
from scipy.spatial.distance import jaccard

## **Backus Naur Form**


### **Production Rules**
&emsp;$S$ &rarr;&nbsp; $(S)$ &nbsp; $O$ &nbsp; $(V)$ &nbsp; $|$ &nbsp; $(S)$ &nbsp; $O$ &nbsp; $(N)$ &nbsp; $|$ &nbsp; $(V)$ &nbsp; $|$ &nbsp; $(N)$

&emsp;$O$ &rarr;&nbsp; $<=living\_neighbours<=$ &nbsp; $|$ &nbsp; $and$ &nbsp; $|$ &nbsp; $or$

&emsp;$V$ &rarr;&nbsp; $is\_alive(cell)$ &nbsp; $|$ &nbsp; $is\_dead(cell)$

&emsp;$N$ &rarr;&nbsp; $0$ &nbsp; $|$ &nbsp; $1$ &nbsp; $|$ &nbsp; $2$ &nbsp; $|$ &nbsp; $3$ &nbsp; $|$ &nbsp; $4$ &nbsp; $|$ &nbsp; $5$ &nbsp; $|$ &nbsp; $6$ &nbsp; $|$ &nbsp; $7$ &nbsp; $|$ &nbsp; $8$

### **Non-Terminal Symbols**

&emsp;$N = \{S, O, V, N\}$

### **Terminal Symbols**

&emsp;$T = \{...all$&nbsp;$others...\}$

### **Starting Symbol**

&emsp;$S$

*Observation*: the action selection $A$ &rarr; &nbsp; $die$ &nbsp; $|$ &nbsp; $survive$ &nbsp; $|$ &nbsp; $become\_alive$ should not be considered as a real production rule since it cannot be reached from the starting symbol. Still it's used to select which action the program should take when the body of the *if statement* is **True**.

## **Individual**

The base game *Conway's Game of Life* comprehends 4 rules. Therefore an individual will be composed by a certain number of rules. A rule is a list of integers, example:

&emsp;$rule$ &rarr; $[3, 5, 21, 38, 16, 19 .....]$

The evaluation of this list of integers is done by using the `generate_rule(array)` function, which can have 2 standard outputs:

*   $['die', '((3)) <=living\_neighbours<= ((4))']$ *(for example)*
*   $['become\_alive', 'abort']$

The first case is a rule which is actually usable since it has *action* => **die** and a *body* => **((3)) <=living\_neighbours<= ((4))**.

The second case arises in the case if `counter > max_iteration_for_expr_creation` which basically means that rule generation has become too much long therefore it's killed.

## **Initial Setup**

In [ ]:
S = ["(S) O (V)", "(S) O (N)", "(N)", "(V)"]
O = ["<=living_neighbours<=", "and", "or"]
V = ["is_alive(cell)", "is_dead(cell)"]
N = ["0", "1", "2", "3", "4", "5", "6", "7", "8"]

A = ["die", "survive", "become_alive"]

Useful parameters later used in the project.

In [ ]:
num_generation = 5 #max number of generations that the game can perform
max_number_rules = 4 #max number of rules per individual
max_array_size = 15 #max size of the list of integers of a single rule
max_range = 50 #range of integers that can be used in a rule
max_iteration_for_expr_creation = 20 #corner case for which a single rule becomes too large
p_mut = 0.2 #choose between 2 types of deeper mutation with a certain prob.
p_cr = 0.3 #choose between 2 types of crossover with a certain prob.

Here we define *target_matrix*, the pattern or the configuration we would like to obtain while doing each *run* of the simulation.

In [ ]:
grid = [
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 1, 0, 0],
]

target_grid = np.array(grid)
print(target_grid)

[[0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 1 0 0 1 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 1 0 0 1 0]
 [0 0 0 0 1 1 0 0]]


This where we create our *initial_grid*, which will be used by the main program to test the rules. Few random cells of the grid will be set to 1 to have at least some cells with the 1 value.

In [ ]:
another_grid = [
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 1, 0]
]

initial_grid = np.array(another_grid)

print(initial_grid)

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0]]


This function, given a rule (a list of integers), will generate a new list containing an $action$ and an $body$, the main content of a rule. This is where the production rules are applied.

In [ ]:
def generate_rule(array):
    action = A[array[0] % len(A)] #head is chosen here
    i = 0
    counter = 0
    pattern = r"[SVNO]"
    body = "S"
    while any(char in body for char in ["S", "V", "N", "O"]):
        counter += 1
        if counter > max_iteration_for_expr_creation: #case for which the expansion become too big
            return [action, "abort"]
        if i >= len(array): i = 0 #case for which we have a small array of integers therefore we have to reiterate it
        result = re.search(pattern, body)
        index = result.start()
        string = ""

        if body[index] == "S":
            string = S[array[i] % len(S)]
        elif body[index] == "O":
            string = O[array[i] % len(O)]
        elif body[index] == "V":
            string = V[array[i] % len(V)]
        else:
            string = N[array[i] % len(N)]
        body = body[:index] + string + body[index + 1:]
        i += 1
    return [action, body]

## **Game Of Life Class**

This is where we define the **GameOfLife** Class. This class is used to  run the game with the rules passed as parameter in the constructor. With the `run(...)` function the *initial_grid* will be updated by applying the rules using the `apply_rules(...)` function.

Inside the `apply_rules(...)` function we:

1.   check if the $body$ is evaluable using the `is_valid_expression(...)` function
2.   in case of success we extract the $head$ and $body$ of the rule in the variable `extract`
3.   we evaluate the $body$ using the `eval(...)` function
4.   in case we get a **True** evaluation, we proceed to execute the $head$


How `eval()` function works can be easily seen in this example below:

In [ ]:
def function1(value):
    return value == 5

def function2(value):
    return value == 6

print(eval("function1(5) and function2(6)"))

True


In [ ]:
class GameOfLife:
    def __init__(self, initial_grid, target_grid, num_generations, rules):
        self.grid = initial_grid
        self.target_grid = target_grid
        self.num_generations = num_generations
        self.current = 0
        self.best_grid = None
        if check_structure_type(rules) == "single list":
            self.rules = [rules]
        else:
            self.rules = rules

    def count_living_neighbors(self, row, col):
        rows, cols = len(self.grid), len(self.grid[0])
        return sum(self.grid[i][j] for i in range(row - 1, row + 2) for j in range(col - 1, col + 2) if (0 <= i < rows and 0 <= j < cols and (i, j) != (row, col)))

    def apply_rules(self, cell, living_neighbours):
        for rule in self.rules:
            if is_valid_expression(generate_rule(rule)[1]):
                extract = generate_rule(rule)
                if eval(extract[1]):
                    if extract[0] == "die":
                        return 0
                    elif extract[0] == "become_alive":
                        return 1
                    else:
                        return cell #survive
        return cell

    def run(self):
        rows, cols = len(self.grid), len(self.grid[0])

        for _ in range(self.num_generations):
            next_state = [[0] * cols for _ in range(rows)]

            for i in range(rows):
                for j in range(cols):
                    living_neighbors = self.count_living_neighbors(i, j)
                    next_state[i][j] = self.apply_rules(self.grid[i][j], living_neighbors)

            self.grid = copy.deepcopy(next_state)

            best = jaccard_distance(self.target_grid, np.array(self.grid)) #fitness is evaluated after every generation

            if best > self.current:
                self.best_grid = copy.deepcopy(self.grid)
                self.current = best

    def get_best_fitness_through_generations(self):
        return self.current

    def get_best_grid(self):
        return np.array(self.best_grid)

This function has been made to cover the corner case if the user decide that an individual is composed by just 1 rule. In this case we still have to wrap that rule inside a list.

In [ ]:
def check_structure_type(structure): #corner case if we have just 1 rule
    if all(isinstance(sublist, list) for sublist in structure):
        return "list of lists"
    elif isinstance(structure, list):
        return "single list"

Function used to check if the $body$ of the generated rule is actually usable or not. The `try: ... except ... :` block is needed since the `eval()` would return an exception if it finds something not evaluable in the expression. In that case we return **False**, which means that the rule cannot be used. This function is basically used to handle the case $body$ = $abort$.

In [ ]:
def is_valid_expression(expr, cell=1, living_neighbours=1):
    try:
        eval(expr)
        return True
    except Exception as e:
        return False

These 2 functions will be used inside the `eval()` function when we will evaluate the $body$.

In [ ]:
def is_dead(cell):
    return cell == 0

def is_alive(cell):
    return cell == 1

## **Fitness Evaluation**

For the fitness evaluation we want to compare the *initial_grid* (modified in the execution of the `run()` function) with the *target_grid* in each generation in the `run()` function. In this way we are able to control if in any generation we get something closer to the *target_grid*. To check how much similarity there is between these 2 matrixes I've decided to use the ***jaccard distance*** function, which measure the dissimilarity based on the proportion of positions where either matrix has a 1

In [ ]:
def jaccard_distance(grid1, grid2):
    return (1 - jaccard(grid1.flatten(), grid2.flatten())) #we don't want the dissimilarity, but we want the similarity

In [ ]:
def fit(rules):
    init_grid = copy.deepcopy(initial_grid)
    gameoflife = GameOfLife(init_grid, target_grid, num_generation, rules)
    gameoflife.run()
    return gameoflife.get_best_fitness_through_generations()

## **Crossover**


For crossover I've chosen to use one-point crossover and uniform crossover between 2 individuals.

In [ ]:
def one_point_crossover(indv1, indv2):
    index = random.randint(0, min(len(indv1) - 1, len(indv2) - 1))
    offspring1 = indv1[:index] + indv2[index:]
    offspring2 = indv2[:index] + indv1[index:]
    return offspring1, offspring2

def uniform_crossover(indv1, indv2):
    offspring = []
    for gene1, gene2 in zip(indv1, indv2):
        selected_gene = random.choice([gene1, gene2])
        offspring.append(selected_gene)
    return offspring

In [ ]:
x = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
y = [[1,1],[1,1],[1,1],[1,1],[1,1],[1,1]]

#examples of usage
print(one_point_crossover(x, y))
print(uniform_crossover(x, y))

([[0, 0], [0, 0], [1, 1], [1, 1], [1, 1], [1, 1]], [[1, 1], [1, 1], [0, 0], [0, 0], [0, 0], [0, 0]])
[[0, 0], [0, 0], [0, 0], [0, 0], [1, 1], [0, 0]]


## **Mutation**


For mutation I've decided to implement 3 types of mutation:

1.   Substitute an existing rule with a new one
2.   For every rule of an individual substitute an integer in the list with a new integer
3.   Pair each rule in an individual following the pattern $i, i + 1$ and perform one-point crossover between the pairs

In [ ]:
def mutation(indv1):
    index = random.randint(0, len(indv1) - 1)
    rule = [random.randint(0, max_range) for _ in range(random.randint(5, max_array_size))]
    indv1[index] = rule
    return indv1

def deeper_number_mutation(indv1):
    for rule in indv1:
        index = random.randint(0, len(rule) - 1)
        rule[index] = random.randint(0, max_range)
    return indv1

def deeper_swap_mutation(indv1):
    pairs = [(indv1[i], indv1[(i + 1) % len(indv1)]) for i in range(len(indv1))]
    offsprings = []
    for x, y in pairs:
        results = one_point_crossover(x, y)
        offsprings.append(random.choice(results))
    return offsprings

In [ ]:
x = [[12,25,80, 5], [11, 17, 93, 70, 60, 23, 57], [72, 28]]
y = [[12,25,80, 5], [11, 17, 93, 70, 60, 23, 57], [72, 28]]
z = [[12,25,80, 5], [11, 17, 93, 70, 60, 23, 57], [72, 28]]

#examples of usage
print(mutation(x))
print(deeper_number_mutation(y))
print(deeper_swap_mutation(z))

[[28, 16, 23, 23, 22, 18, 15], [11, 17, 93, 70, 60, 23, 57], [72, 28]]
[[12, 38, 80, 5], [11, 17, 93, 70, 60, 23, 34], [72, 9]]
[[11, 17, 93, 5], [11, 28], [72, 25, 80, 5]]


## **Selection criteria**


Tournament selection has been chosen for this project.

In [ ]:
def tournament_selection(fit, pop, t_size=5):
  tournament = random.choices(pop, k=t_size)
  return max(tournament, key=fit)

## **Population initialization**

Function used to initialize the population of individuals.

In [ ]:
def init_population(pop_size):
  pop = []
  for i in range(pop_size):
    rules = []
    for j in range(max_number_rules):
        rule = [random.randint(0, max_range) for k in range(random.randint(5, max_array_size))]
        rules.append(rule)
    pop.append(rules)
  return pop

## **GE Algorithm**

In [ ]:
def GE(fit, pop_size, n_iter = 10):
  pop = init_population(pop_size)

  best = []

  for i in range(0, n_iter):
    selected = [tournament_selection(fit, pop) for _ in range(0, pop_size)]
    pairs = [[selected[i], selected[i + 1]] for i in range(len(selected) - 1)]

    offsprings = []
    for x, y in pairs:
        if random.random() < p_cr:
            of1, of2 = one_point_crossover(x, y)
        else:
            of1 = uniform_crossover(x, y)
            of2 = uniform_crossover(x, y)
        offsprings.append(of1)
        offsprings.append(of2)

    if random.random() < p_mut:
        pop = [deeper_number_mutation(mutation(x)) for x in offsprings]
    else:
        pop = [deeper_swap_mutation(mutation(x)) for x in offsprings]

    candidate_best = max(pop, key=fit)
    print("Best candidate rules:")
    for rule in candidate_best:
      print(f"{generate_rule(rule)},")
    print(f"candidate fitness: {fit(candidate_best)} , best fitness: {fit(best)}")
    if fit(candidate_best) > fit(best):
      best = candidate_best
    print(f"Best fitness at generation {i}: {fit(best)} \n")
  return best

In [ ]:
best = GE(fit, 100)
print("BEST RULES FOUND\n")
for rule in best:
    print(f"{generate_rule(rule)},")

init_grid_re = copy.deepcopy(initial_grid)
gameoflife_re = GameOfLife(init_grid_re, target_grid, num_generation, best)
gameoflife_re.run()
print(f"\nInitial grid:\n{init_grid_re}\nTarget grid:\n{target_grid}\nBest grid throughout {num_generation} generations:\n{gameoflife_re.get_best_grid()}")


Best candidate rules:
['become_alive', '((is_dead(cell))) <=living_neighbours<= (0)'],
['become_alive', '((is_dead(cell))) <=living_neighbours<= (7)'],
['become_alive', '(is_alive(cell))'],
['survive', '((is_dead(cell))) and (6)'],
candidate fitness: 0.32258064516129037 , best fitness: 0
Best fitness at generation 0: 0.32258064516129037 

Best candidate rules:
['die', '((((is_dead(cell))) or (is_alive(cell))) <=living_neighbours<= (is_alive(cell))) <=living_neighbours<= (is_alive(cell))'],
['die', '((((is_dead(cell))) or (is_alive(cell))) <=living_neighbours<= (is_alive(cell))) or (is_alive(cell))'],
['die', '(is_alive(cell))'],
['become_alive', '(is_dead(cell))'],
candidate fitness: 0.37037037037037035 , best fitness: 0.32258064516129037
Best fitness at generation 1: 0.37037037037037035 

Best candidate rules:
['become_alive', '((1)) <=living_neighbours<= (is_dead(cell))'],
['die', '((is_alive(cell))) or (5)'],
['die', '((is_alive(cell))) or (1)'],
['survive', 'abort'],
candidate fitn

## **Variation of the Production Rules**

The production rules that have been used up to now have been created by analyzing how the real rules of the Conway's Game of Life were made. But since with those rules it's very difficult to obtain certain patterns in a matrix, I've decided to try using a different set of production rules for which we take into account how the **neighbours cell are arranged**.

In [ ]:
S = ["(S) O (V)", "(S) O (N)", "(N)", "(V)"]
O = ["and", "or"]
V = ["is_alive(cell)", "is_dead(cell)"]
N = ["n_alive(celln)", "not n_alive(celln)", "s_alive(cells)", "not s_alive(cells)", "w_alive(cellw)", "not w_alive(cellw)", "e_alive(celle)", "not e_alive(celle)",
     "nw_alive(cellnw)", "not nw_alive(cellnw)", "ne_alive(cellne)", "not ne_alive(cellne)", "sw_alive(cellsw)", "not sw_alive(cellsw)", "se_alive(cellse)", "not se_alive(cellse)",]
A = ["die", "survive", "become_alive"]

As done before with the `is_alive(cell)` and `is_dead(cell)` functions, here we have to include all the functions for the 8 positions around a single cell.

In [ ]:
def n_alive(celln): #NORD
    return celln == 1

def s_alive(cells): #SOUTH
    return cells == 1

def w_alive(cellw): #WEST
    return cellw == 1

def e_alive(celle): #EAST
    return celle == 1

def ne_alive(cellne): #NORD-EAST
    return cellne == 1

def nw_alive(cellnw): #NORD-WEST
    return cellnw == 1

def se_alive(cellse): #SOUTH-EAST
    return cellse == 1

def sw_alive(cellsw): #SOUTH-WEST
    return cellsw == 1

Also the **GameOfLife** class has to be modified since now we are not interested into `count_living_neighbors()` but we are interested into `get_neighbours_values()`. Also the `apply_rules()` function has to be modified since now we have to pass all the values of the neighbour of a cell.

*Observation*: to avoid annoying corner cases in the border of the grid, when we scan the grid to apply the rules, we scan all the cells that are in the rage of `(1, rows - 2) x (1, cols - 2)`

In [ ]:
class GameOfLife:
    def __init__(self, initial_grid, target_grid, num_generations, rules):
        self.grid = initial_grid
        self.target_grid = target_grid
        self.num_generations = num_generations
        self.current = 0
        self.best_grid = None
        if check_structure_type(rules) == "single list":
            self.rules = [rules]
        else:
            self.rules = rules

    def get_neighbours_values(self, row, col):
        rows, cols = len(self.grid), len(self.grid[0])
        return [self.grid[i][j] for i in range(row - 1, row + 2) for j in range(col - 1, col + 2) if (0 <= i < rows and 0 <= j < cols and (i, j) != (row, col))]
        #  1  2  3
        #  4     5
        #  6  7  8

    def apply_rules(self, cell, cellnw, celln, cellne, cellw, celle, cellsw, cells, cellse):
        for rule in self.rules:
            if is_valid_expression(generate_rule(rule)[1]):
                extract = generate_rule(rule)
                if eval(extract[1]):
                    if extract[0] == "die":
                        return 0
                    elif extract[0] == "become_alive":
                        return 1
                    else:
                        return cell #survive
        return cell

    def run(self):
        rows, cols = len(self.grid), len(self.grid[0])

        for _ in range(self.num_generations):
            next_state = [[0] * cols for _ in range(rows)]

            for i in range(1, rows - 2):
                for j in range(1, cols - 2):
                    nb = self.get_neighbours_values(i, j)
                    next_state[i][j] = self.apply_rules(self.grid[i][j], nb[0], nb[1], nb[2], nb[3], nb[4], nb[5], nb[6], nb[7])

            self.grid = copy.deepcopy(next_state)

            best = jaccard_distance(self.target_grid, np.array(self.grid)) #fitness is evaluated after every generation

            if best > self.current:
                self.best_grid = copy.deepcopy(self.grid)
                self.current = best

    def get_best_fitness_through_generations(self):
        return self.current

    def get_best_grid(self):
        return np.array(self.best_grid)


In [ ]:
#redefinition of others initial and target grids
grid = [
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0],
]

target_grid = np.array(grid)

another_grid = [
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 0, 0, 0]
]

initial_grid = np.array(another_grid)

In [ ]:
best = GE(fit, 100)
print("BEST RULES FOUND\n")
for rule in best:
    print(f"{generate_rule(rule)},")

init_grid_re = copy.deepcopy(initial_grid)
gameoflife_re = GameOfLife(init_grid_re, target_grid, num_generation, best)
gameoflife_re.run()
print(f"\nInitial grid:\n{init_grid_re}\nTarget grid:\n{target_grid}\nBest grid throughout {num_generation} generations:\n{gameoflife_re.get_best_grid()}")

Best candidate rules:
['become_alive', '((is_dead(cell))) and (w_alive(cellw))'],
['survive', '(not s_alive(cells))'],
['become_alive', '((is_alive(cell))) or (is_alive(cell))'],
['become_alive', '((is_dead(cell))) and (nw_alive(cellnw))'],
candidate fitness: 0.23076923076923073 , best fitness: 0.11111111111111116
Best fitness at generation 0: 0.23076923076923073 

Best candidate rules:
['die', '((((is_dead(cell))) and (not se_alive(cellse))) and (ne_alive(cellne))) or (is_alive(cell))'],
['die', '((is_alive(cell))) or (is_alive(cell))'],
['survive', '((not e_alive(celle))) or (not e_alive(celle))'],
['become_alive', '(is_dead(cell))'],
candidate fitness: 0.375 , best fitness: 0.23076923076923073
Best fitness at generation 1: 0.375 

Best candidate rules:
['become_alive', '((not n_alive(celln))) or (w_alive(cellw))'],
['die', '((s_alive(cells))) and (is_dead(cell))'],
['become_alive', '(((is_alive(cell))) or (ne_alive(cellne))) or (s_alive(cells))'],
['become_alive', '((not se_alive(ce